In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

In [2]:
logs_path = Path('./log/')

In [3]:
re_baseline_total = re.compile("'runBaseline', 'time': ")
re_baseline = re.compile("'check_all_snapshots', 'time': \d*\.\d*")

re_loadmodel = re.compile("'get_model', 'time': \d*\.\d*")
re_preprocess = re.compile("'preprocess', 'time': \d*\.\d*")
re_get_cosine_sims = re.compile("'get_cosine_sims', 'time': \d*\.\d*")
re_get_pixel_sims = re.compile("'get_pixel_sims', 'time': \d*\.\d*")
re_get_struc_sims = re.compile("'get_struc_sims', 'time': \d*\.\d*")
re_get_mean_sq_errs = re.compile("'get_mean_squared_errors', 'time': \d*\.\d*")
re_check_snapshot = re.compile("'check_snapshot', 'time': \d*\.\d*")
re_run = re.compile("'run', 'time': \d*\.\d*")

re_time = re.compile("\d*\.\d*")

In [35]:
baseline_times = []
baseline_get_pixel_times = []
baseline_get_struc_times = []
baseline_get_mse_times = []
preprocess_times = []
get_cosine_times = []
get_pixel_times = []
get_struc_times = []
get_mse_times = []

def strip_times(time_str_list):
    time_list = []
    for t in time_str_list:
        time_list.append(float(re_time.search(t).group()))
    return time_list

for log_file_path in logs_path.glob("*.log"):
    if "bg_black" not in log_file_path.name and "baseline" not in log_file_path.name:
        continue
    with open(log_file_path) as f:
        log_file = f.read()
    if "baseline" in log_file_path.name:
        # idx_time = re_baseline.search(log_file).end()
        idx_time_total = re_baseline_total.search(log_file).end()
        time_baseline_total = float(log_file[idx_time_total:idx_time_total+10])
        baseline_times += strip_times(re_baseline.findall(log_file))
        baseline_get_pixel_times += strip_times(re_get_pixel_sims.findall(log_file))
        baseline_get_struc_times += strip_times(re_get_struc_sims.findall(log_file))
        baseline_get_mse_times += strip_times(re_get_mean_sq_errs.findall(log_file))
    else:
        preprocess_times += strip_times(re_preprocess.findall(log_file))
        get_cosine_times += strip_times(re_get_cosine_sims.findall(log_file))
        get_pixel_times += strip_times(re_get_pixel_sims.findall(log_file))
        get_struc_times += strip_times(re_get_struc_sims.findall(log_file))
        get_mse_times += strip_times(re_get_mean_sq_errs.findall(log_file))

### baseline

In [36]:
print(np.average(baseline_get_pixel_times)) # per snapshot

0.05681246566772461


In [37]:
print(np.average(baseline_get_mse_times)) # per snapshot

0.019621464252471923


In [38]:
print(np.average(baseline_get_struc_times)) # per snapshot

0.3227230306625366


In [39]:
baseline_timings = {
    "get_pix": np.average(baseline_get_pixel_times),
    "get_mse": np.average(baseline_get_mse_times),
    "get_struc": np.average(baseline_get_struc_times),
    "per_ss": np.average(baseline_times) / 10,
}

pd.DataFrame(baseline_timings, index=["baseline"]).T

,baseline
get_pix,0.056812
get_mse,0.019621
get_struc,0.322723
per_ss,0.400045


### ours

In [40]:
np.average(preprocess_times) # per object

0.12204192591181207

In [48]:
# 10 snapshots per test case
# 10 runs per test case
# 25 test cases in total
np.sum(preprocess_times) / (10 * 10 * 25) # per snapshot

3.1355011605262755

In [49]:
np.average(get_pixel_times) # per object

0.06289765396118165

In [50]:
# 10 runs per test case
# 25 test cases in total
# 3 different sets of experiments (diff background colour)
np.sum(get_pixel_times) / (10 * 25 * 3) # per snapshot

0.20965884653727213

In [51]:
np.average(get_mse_times) # per snapshot

0.1911738235473633

In [52]:
np.average(get_struc_times) # per snapshot

2.4963019045829773

In [53]:
np.average(get_cosine_times) # per snapshot

0.17411484260559082

In [55]:
main_timings = {
    "preprocess": np.sum(preprocess_times) / (10 * 10 * 25),
    "get_pix": np.average(get_pixel_times),
    "get_mse": np.average(get_mse_times),
    "get_struc": np.average(get_struc_times),
    "get_cos": np.average(get_cosine_times),
}

pd.DataFrame(main_timings, index=["main"]).T

,main
preprocess,3.135501
get_pix,0.062898
get_mse,0.191174
get_struc,2.496302
get_cos,0.174115


---